## **FINM 32000 - HW1**

#### Yumeng Zhang (12372205)

I discussed Problem 1 with Yitong Li, and Problem 2 with Shuangming Shi.

In [3]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq

# Problem 1

##### **(a)**

In [4]:
class UpAndOutPut:
    
    def __init__(self, K, T, barrier, observationinterval):
        
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

In [19]:
# hw1contract = UpAndOutPut(K=95, T=0.25, barrier=107, observationinterval=0.02)
hw1contract = UpAndOutPut (K=95, T = 0.25, barrier = 114, observationinterval = 0.02)

In [20]:
class GBMdynamics: 
    
    def __init__(self, S, r, rGrow, sigma=None):

        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma
        
    def update_sigma(self, sigma):
        
        self.sigma = sigma
        return self

In [21]:
hw1dynamics = GBMdynamics(S = 100, sigma = 0.4, rGrow = 0, r = 0)

In [22]:
class Tree:
    
    def __init__(self, N):
        
        self.N = N
                
    def price_upandout(self, dynamics, contract): 
        
        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)
        
        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)  
        # Here I decided to make the SMALLER indexes in this array correspond to HIGHER S
        
        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")
            
        nu = dynamics.r - (dynamics.sigma**2)/2                                                          # complete this 
        Pu = 0.5*(((dynamics.sigma**2)*deltat+(nu**2)*(deltat**2))/(deltax**2) + nu*deltat/deltax)       # complete this
        Pd = 0.5*(((dynamics.sigma**2)*deltat+(nu**2)*(deltat**2))/(deltax**2) - nu*deltat/deltax)       # complete this
        Pm = 1 - ((dynamics.sigma**2)*deltat+(nu**2)*(deltat**2))/(deltax**2)                            # complete this

        optionprice = np.maximum(contract.K-Sgrid,0)   # an array of time-T option prices.

        barrier_check = []

        for i in range(len(Sgrid)):

            if Sgrid[i] >= contract.barrier:
                barrier_check.append(0)
            else:
                barrier_check.append(1)
            

        # Next, induct backwards to time 0, updating the optionprice array 
        # Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
    
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:

            # insert lines of code here if needed

            Cu = Pu * optionprice[:-2]
            Cm = Pm * optionprice[1:-1]
            Cd = Pd * optionprice[2:]

            optionprice = np.exp((-1)*dynamics.r*deltat) * (Cu+Cm+Cd)  # complete this

            barrier_check = barrier_check[1:-1]

            if abs(t/contract.observationinterval-round(t/contract.observationinterval)) < 1e-8:  
                optionprice = optionprice * barrier_check
    

        return optionprice[0]  
         
        # The [0] is assuming that we are shrinking the optionprice array in each iteration of the loop,
        # until finally there is only 1 element in the array.
        # If instead you are keeping unchanged the size of the optionprice array in each iteration, 
        # then you need to change the [0] to a different index.
        

In [38]:
hw1tree = Tree (N = 10000)

price_tree = hw1tree.price_upandout(hw1dynamics, hw1contract)

print("Using a trinomial tree and with barrier conditions, the option price at time 0 is " + str(round(price_tree, 4)) + ".")

Using a trinomial tree and with barrier conditions, the option price at time 0 is 5.3011.


In [39]:
# to remove barrier conditions in trinomial tree method, set the barrier to a large value, such as 10000
# so that S will always be under the barrier

hw1contract_ = UpAndOutPut (K = 95, T = 0.25, barrier = 10000, observationinterval = 0.02)
hw1tree_ = Tree (N = 10000)

price_tree_ = hw1tree_.price_upandout(hw1dynamics, hw1contract_)

print("Remove the barrier conditions, trinomial tree method shows that the option price at time 0 is " + str(round(price_tree_, 4)) + ".")

Remove the barrier conditions, trinomial tree method shows that the option price at time 0 is 5.5195.


In [100]:
# Black-Scholes

from scipy.stats import norm

def BS_call(S, K, T, r, sigma):

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    return price

def BS_put(S, K, T, r, sigma):

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return price


price_bs = BS_put(S = 100, K = 95, T = 0.25, r = 0, sigma = 0.4)

print("Using Black-Scholes to price Europeans, the option price at time 0 is " + str(round(price_bs, 4)) + ".")

Using Black-Scholes to price Europeans, the option price at time 0 is 5.5195.


Trinomial tree method with barrier conditions removed gives the same time-0 option price as Black-Scholes, which proves that it has priced Europeans correctly.

##### **(b)**

In [40]:
price_out = price_tree
price_in = price_bs - price_out

print("The time-0 price of the up-and-in put is " + str(round(price_in, 4)) + ".")

The time-0 price of the up-and-in put is 0.2185.


##### **(c1)**

The time-0 price of the continuously-monitored barrier option is smaller than the time-0 price of the discretely-monitored option, because it is more likely to knock out the put option, and therefore less likely to expire. 

##### **(c2)**

The continuously monitored barrier option can be replicated by longing 1 plain vanilla put struck at 95, and shorting α plain vanilla calls struck at 136.8. Let the P = price of one plain vanilla put, C = price of one plain vanilla call.

At the time when S is at the barrier, P - α * C = 0, α = P / C.

According to BS formula, P = K * np.exp(-r * T) * N(-d2) - S * N(-d1), C = S * N(d1) - K * np.exp(-r * T) * N (d2).

Plug in, and we can get P = 95 * N(-d2) - S * N(-d1) = 95 * N(d1) - S * N(d2), C = S * N(d1) - 136.8 * N(d2).

When S is at the barrier, S = 114.

95 * N(d1) - 114 * N(d2) = α * (114 * N(d1) - 136.8 * N(d2))

(114 * α - 95) * N(d1) = (136.8 * α - 100) * N(d2)

Therefore, 114 * α - 95 = 0, 136.8 * α - 100 = 0.

α = 95/114 = 100/136.8 = 0.83333

In [55]:
alpha = 95/114

vanilla_put = BS_put(S = 100, K = 95, T = 0.25, r = 0, sigma = 0.4)
vanilla_call = BS_call(S = 100, K = 136.8, T = 0.25, r = 0, sigma = 0.4)

price_continuous = vanilla_put - alpha * vanilla_call

print("The time-0 value of the continuously-monitored barrier option is " + str(round(price_continuous,4)) + ".")

The time-0 value of the continuously-monitored barrier option is 5.0315.


# Problem 2

In [ ]:
# uses the same GBMdynamics class as in Problem 1

In [101]:
class CallOption:
    
    def __init__(self, K, T, price=None):

        self.K = K
        self.T = T
        self.price = price

    def BSprice(self, dynamics):
        # ignores self.price if given, because this function calculates price based on the dynamics 
        
        F = dynamics.S*np.exp(dynamics.rGrow*self.T)
        sd = dynamics.sigma*np.sqrt(self.T)
        d1 = np.log(F/self.K)/sd+sd/2
        d2 = d1-sd
        return np.exp(-dynamics.r*self.T)*(F*norm.cdf(d1)-self.K*norm.cdf(d2))

    def BSprice_f(self, S, K, T, r,rGrow, sigma):

        # this function the same as BSprice, but is used to solving for sigma in self.IV

        F = S*np.exp(rGrow*T)
        sd = sigma*np.sqrt(T)
        d1 = np.log(F/K)/sd+sd/2
        d2 = d1-sd

        return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))

    def IV(self, dynamics):
        
        # ignores dynamics.sigma, because this function solves for sigma.  
        
        if self.price is None: 
            raise ValueError('Contract price must be given')
    
        df = np.exp(-dynamics.r*self.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-self.K)*df])
        C = self.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan 

        dytry = dynamics
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        dytry.sigma = 0.2
        while self.BSprice(dytry)>C:
            dytry.sigma /= 2
        while self.BSprice(dytry)<C:
            dytry.sigma *= 2
            
        hi = dytry.sigma
        lo = hi/2

        # We have calculated "lo" and "hi" which bound the implied volatility from below and above. 
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval, 
        # for purposes of this homework, you may either (A) write your own bisection algorithm, 
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution 
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).  
        # However, if you are able to bound the solution (of a single-variable problem), 
        # then bisection or Brent will be more reliable.

        # you fill this in, using bisect or brentq imported from scipy.optimize, or by writing your own bisection algorithm.
        
        impliedVolatility = bisect(lambda x: self.BSprice_f(dynamics.S, self.K, self.T, dynamics.r, dynamics.rGrow, x) - C, lo, hi)     
        
        return impliedVolatility


In [102]:
#Test the BSprice function
dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract2 = CallOption(K=100, T=0.5)
contract2.BSprice(dynamics2)

11.246291601828489

In [103]:
#Test the IV function
contract2.price = 12
contract2.IV(dynamics2)

0.427005424113304

##### **(a)**

In [104]:
# 0.5 year

contract_a1 = CallOption (K = 100, T = 0.5, price = 11.25)
dynamics_a1 = GBMdynamics(S = 100, r = 0, rGrow = 0)
vol_a1 = contract_a1.IV(dynamics_a1)

print("The time-0 Black-Scholes implied volatility of 0.5-expiry call is " + str(round(vol_a1,4)) + ".")

The time-0 Black-Scholes implied volatility of 0.5-expiry call is 0.4001.


In [98]:
# 1 year

contract_a2 = CallOption (K = 100, T = 1, price = 12)
dynamics_a2 = GBMdynamics(S = 100, r = 0, rGrow = 0)
vol_a2 = contract_a2.IV(dynamics_a2)

print("The time-0 Black-Scholes implied volatility of 1-expiry call is " + str(round(vol_a2,4)) + ".")

The time-0 Black-Scholes implied volatility of 1-expiry call is 0.3019.


##### **(b)**

In [99]:
vol_b = 0.5* (vol_a1 + vol_a2)
price_b = BS_call(S = 100, K = 100, T = 0.75, r = 0, sigma = vol_b)

print("The time-0 price of the 0.75-expiry call is " + str(round(price_b,4)) + ".")

The time-0 price of the 0.75-expiry call is 12.0815.


##### **(c)**

To get an arbitrage, we build the portfolio by longing one unit of 1-expiry call and shorting one unit of 0.75-expiry call.

**Time 0**

According to (a) and (b), the time-0 price of 1-expiry call is 12, the time-0 price of 0.75-expiry call is 12.0815. 

Therefore, V_0 = 12 - 12.0815 = -0.0815 < 0.

**Time 0.75**

If S_0.75 > 100, short one share of stock and get K.

**Time 1**

If S_1 > 100 and S_0.75 > 100, exercise the long 1-expiry call, close the short position of stock which was opened at time 0.75, V_1 = 1.

If S_1 > 100 and S_0.75 <= 100, exercise the long 1-expiry call, V_1 = S_1 - K > 0.

If S_1 <= 100 and S_0.75 > 100, V_1 = K - S_1 > 0.

If S_1 <= 100 and S_0.75 <= 100, V_1 = 0.

In this way, we have a type two arbitrage.